# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [6]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

sum_gen = 0
n = 0
for sqrt_value in generator:
    n = n + 1
    print(str(n) + '. '+ str(sqrt_value))
    sum_gen = sum_gen + sqrt_value
    if n == 5:
        print('limit 5 sum_gen = ' + str(sum_gen))

1. 1.0
2. 1.4142135623730951
3. 1.7320508075688772
4. 2.0
5. 2.23606797749979
limit 5 sum_gen = 8.382332347441762
6. 2.449489742783178
7. 2.6457513110645907
8. 2.8284271247461903
9. 3.0
10. 3.1622776601683795
11. 3.3166247903554
12. 3.4641016151377544
13. 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [7]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [8]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [9]:
# to do: homework :)
import dlt

p_pipeline = dlt.pipeline(destination='duckdb', dataset_name='all_people')

info = p_pipeline.run(people_1(),
                             table_name="all_people",
                             write_disposition="replace")

print(info)

info = p_pipeline.run(people_2(),
                             table_name="all_people",
                             write_disposition="append")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.35 seconds
1 load package(s) were loaded to destination duckdb and into dataset all_people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1706983576.58903 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.27 seconds
1 load package(s) were loaded to destination duckdb and into dataset all_people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1706983577.3865733 is LOADED and contains no failed jobs


In [30]:
import duckdb

conn = duckdb.connect(f"{p_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{p_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n all_people table below:")

people = conn.sql("SELECT * FROM all_people").df()
display(people)

print("\n\n\n stream_download table below:")

sum_age = conn.execute("SELECT SUM(age) as s FROM all_people").df()##.fetchone()[0]
print(sum_age)
print(f"\n\nSum of ages from people_1 and people_2 generators is : {sum_age['s'].iloc[0].astype(int)}")


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ all_people          │
└─────────────────────┘




 all_people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1706983576.58903,yCTKiOdVAS7t9g,None
1,2,Person_2,27,City_A,1706983576.58903,RH9RScg1VarKtw,None
2,3,Person_3,28,City_A,1706983576.58903,7k8mQ2DYYON2jw,None
3,4,Person_4,29,City_A,1706983576.58903,eX4Kue9B4TcoQQ,None
4,5,Person_5,30,City_A,1706983576.58903,CRy5D1tUNu2zNw,None
5,3,Person_3,33,City_B,1706983577.3865733,HTh/HRyqcgmkdw,Job_3
6,4,Person_4,34,City_B,1706983577.3865733,iuisiXpBn35WLA,Job_4
7,5,Person_5,35,City_B,1706983577.3865733,ptul/Uyj2o3jzw,Job_5
8,6,Person_6,36,City_B,1706983577.3865733,iLFMO0spFtl5fg,Job_6
9,7,Person_7,37,City_B,1706983577.3865733,juPpMkYebGxQ9g,Job_7





 stream_download table below:
       s
0  353.0


Sum of ages from people_1 and people_2 generators is : 353


In [31]:
pm_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_merged')

info = pm_pipeline.run(people_1(),
                             table_name="people_merge",
                             write_disposition="replace",
                             primary_key="ID")

print(info)

info = pm_pipeline.run(people_2(),
                             table_name="people_merge",
                             write_disposition="merge",
                             primary_key="ID")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.38 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_merged
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1706985921.3649104 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.48 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_merged
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1706985922.276094 is LOADED and contains no failed jobs


In [33]:
conn = duckdb.connect(f"{pm_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pm_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_merge table below:")

people = conn.sql("SELECT * FROM people_merge order by id").df()
display(people)

print("\n\n\n stream_download table below:")

sum_age = conn.execute("SELECT SUM(age) as s FROM people_merge").df()##.fetchone()[0]
print(sum_age)
print(f"\n\nSum of ages from people_1 and people_2 generators is : {sum_age['s'].iloc[0].astype(int)}")

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_merge        │
└─────────────────────┘




 people_merge table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1706985921.3649104,Dr7Mt0G/WhwRXw,None
1,2,Person_2,27,City_A,1706985921.3649104,3MoKRe49oqJpOg,None
2,3,Person_3,33,City_B,1706985922.276094,eCiRZuNT/vFjvQ,Job_3
3,4,Person_4,34,City_B,1706985922.276094,uApq0DbBEf/kmQ,Job_4
4,5,Person_5,35,City_B,1706985922.276094,Y6BgRyeePVFG2Q,Job_5
5,6,Person_6,36,City_B,1706985922.276094,8z9DWfyekjSmGA,Job_6
6,7,Person_7,37,City_B,1706985922.276094,Lut1dr4NHzsudw,Job_7
7,8,Person_8,38,City_B,1706985922.276094,artpljKK+gbJTQ,Job_8





 stream_download table below:
       s
0  266.0


Sum of ages from people_1 and people_2 generators is : 266


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX